<a href="https://colab.research.google.com/github/rasmusli/EDAN70-relationextraction/blob/master/BioBERTEnvironment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BioBERT Environment Setup

This script sets up everthing needed to run BioBERT on Colab. First we mount our drive where all the files needed for the setup are located. This requires an authorization-process.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!git clone https://github.com/dmis-lab/biobert.git

fatal: destination path 'biobert' already exists and is not an empty directory.


In [2]:
!pip install -r biobert/requirements.txt

All requirements downloaded. Next step is to install CUDA. Specifically version 9.0 for compability with Tensorflow.

In [3]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo dpkg -i /content/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install cuda-9-0 #REMEMBER YOU HAVE TO ANSWER THE PROMPT!

--2020-04-27 11:28:40--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?R7fmJJGEwBD9dAWlSbXsSUFyksfe_wXimSF1Qt3HkvwJ7mq9Sf-oGnEwC6eqJ7ajULM1PmeKZE3qwgG5nWhcupaBHhT3Xu5kZcz1EpMVG-MiuxHkSdTBcoNIa0X7KlbIIQLrQ2pRcjSY9rGXhvc_5B6kEkeoZaM067nzuwe4vdSnDLCMugNkstus7vH7TXDy_mTed8-H-PL_6ooCTSVZ [following]
--2020-04-27 11:28:40--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?R7fmJJGEwBD9dAWlSbXsSUFyksfe_wXimSF1Qt3HkvwJ7mq9Sf-oGnEwC6eqJ7ajULM1PmeKZE3qwgG5nWhcupaBHhT3Xu5kZcz1EpMVG-MiuxHkSd

Next we need to retrieve the pretrained weights and datasets to be able to fine tune BioBERT. Both of these are stored on our drive.

In [15]:
!tar -xvf drive/My\ Drive/EDAN70/Data/biobert_v1.1_pubmed.tar.gz

biobert_v1.1_pubmed/
biobert_v1.1_pubmed/model.ckpt-1000000.data-00000-of-00001
biobert_v1.1_pubmed/model.ckpt-1000000.meta
biobert_v1.1_pubmed/bert_config.json
biobert_v1.1_pubmed/vocab.txt
biobert_v1.1_pubmed/model.ckpt-1000000.index


In [37]:
!unzip drive/My\ Drive/EDAN70/Data/REdata.zip
!unzip drive/My\ Drive/EDAN70/Data/NERdata.zip


Archive:  drive/My Drive/EDAN70/Data/REdata.zip
replace euadr/10/dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  drive/My Drive/EDAN70/Data/NERdata.zip
   creating: BC2GM/
  inflating: BC2GM/devel.tsv         
  inflating: BC2GM/test.tsv          
  inflating: BC2GM/train.tsv         
  inflating: BC2GM/train_dev.tsv     
   creating: BC4CHEMD/
  inflating: BC4CHEMD/devel.tsv      
  inflating: BC4CHEMD/test.tsv       
  inflating: BC4CHEMD/train.tsv      
  inflating: BC4CHEMD/train_dev.tsv  
   creating: BC5CDR-chem/
  inflating: BC5CDR-chem/devel.tsv   
  inflating: BC5CDR-chem/test.tsv    
  inflating: BC5CDR-chem/train.tsv   
  inflating: BC5CDR-chem/train_dev.tsv  
   creating: BC5CDR-disease/
  inflating: BC5CDR-disease/devel.tsv  
  inflating: BC5CDR-disease/test.tsv  
  inflating: BC5CDR-disease/train.tsv  
  inflating: BC5CDR-disease/train_dev.tsv  
   creating: JNLPBA/
  inflating: JNLPBA/devel.tsv        
  inflating: JNLPBA/test.tsv         
  inflating: JNLPBA/

### Model fine tuning

In [0]:
import os
os.environ['BIOBERT_DIR']= './biobert_v1.1_pubmed' #Pre-trained model.
os.environ['RE_DIR'] = './GAD/2' # Dataset. Could be any dataset you want to use.
os.environ['OUTPUT_DIR'] = './RE_outputs2_10epochs' # Output
!mkdir $OUTPUT_DIR

In [33]:
#CHECK HOW MUCH GPU COLAB IS GIVING YOU
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.7 GB  | Proc size: 114.0 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [34]:
!python biobert/run_re.py --task_name=gad --do_train=true --do_eval=true --do_predict=true --vocab_file=$BIOBERT_DIR/vocab.txt --bert_config_file=$BIOBERT_DIR/bert_config.json --init_checkpoint=$BIOBERT_DIR/model.ckpt-1000000 --max_seq_length=128 --train_batch_size=32 --learning_rate=2e-5 --num_train_epochs=10.0 --do_lower_case=false --data_dir=$RE_DIR --output_dir=$OUTPUT_DIR

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

To evaluate the results we can run the re_eval.py code from the BioBERT model.

In [35]:
!python biobert/biocodes/re_eval.py --output_path=$OUTPUT_DIR/test_results.tsv --answer_path=$RE_DIR/test.tsv


f1 score    : 81.55%
recall      : 82.86%
precision   : 80.28%
specificity : 77.47%


### NER

In [0]:
def fine_tune_NER(data_path):
  os.environ['BIOBERT_DIR']= './biobert_v1.1_pubmed' #Pre-trained model.
  os.environ['NER_DIR'] = data_path # Dataset. Could be any dataset you want to use.
  os.environ['OUTPUT_DIR'] = './ner_outputs' # Output
  !mkdir $OUTPUT_DIR
  !python biobert/run_ner.py --do_train=true --do_eval=true --vocab_file=$BIOBERT_DIR/vocab.txt --bert_config_file=$BIOBERT_DIR/bert_config.json --init_checkpoint=$BIOBERT_DIR/model.ckpt-1000000 --num_train_epochs=10.0 --data_dir=$NER_DIR --output_dir=$OUTPUT_DIR


In [39]:
fine_tune_NER('./BC5CDR-chem')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 